# Retention: Comparing kernel to current redshift pulls
* StelllarAlgo Data Science
* Ryan Kazmerik & Grant Donst
* Apr 3, 2023

In [1]:
import awswrangler as wr
import pandas as pd

from data_sci_toolkit.aws_tools import permission_tools 
from data_sci_toolkit.aws_tools import redshift_tools

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

/opt/miniconda3/envs/stellar/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

DFBQ-VMJG
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/
Authorized as AROAQBUNWK7XN5DTYWPLJ:rkazmerik@stellaralgo.com


In [3]:
DATABASE = "integrations_milb_aviators"
S3_BUCKET = "s3://dev-athena-testing-us-east-1"

In [4]:
df1 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        WITH customers as (
            select * 
            from integrations_milb_aviators.customers c
            CROSS JOIN UNNEST(account_ids['ticketing'], account_ids['retail']) as t(ticket_ids, retail_ids)
        ),
        ticketing AS (
            select *
            from integrations_milb_aviators.tickets t
            CROSS JOIN UNNEST(resales, forwards, scans) as t(resale_data, forward_data, scan_data)
            where returned_at IS NULL
        )

        select * 
        from ticketing t
        INNER JOIN integrations_milb_aviators.ticketing_events e
        on t.event_id = e.id
        INNER JOIN customers c
        on c.ticket_ids = t.account_id
    """
)
df1.shape

(2357098, 76)

In [9]:
df1.head(5)

,source_system,source_system_group,stlr_reported_revenue,client_property,stlr_product,seat_id,section_name,row_name,seat_num,season_id,event_id,event_name,plan_event_id,ticket_type,comp_code,promo_code,account_id,price_code,price,account_rep_id,account_rep_name,purchased_at,returned_at,updated_at,forwards,resales,scans,resale_data,forward_data,scan_data,source_system.1,source_system_group.1,id,stlr_is_renewal,stlr_game_number,client_property.1,name,full_name,venue,team,start_date,end_date,day_of_week,time,time_of_day,event_type,season_id.1,season,season_year,game_number,min_events,total_events,fse,has_payment_plan,major_category,minor_category,id.1,account_ids,email,first_name,last_name,address,city,state,country,zip,phone,company_name,account_type,account_rep_name.1,demographics,scv_history,scv_updated_at,venues,ticket_ids,retail_ids
0,ticketmaster,ticketing,0.0,milb_aviators,Comp,7EC10929C99B124C89D9EBD59267B37D,R18,V,8,23,1615,EVLB0317,<NA>,Adult,1,<NA>,280883,DC,0.0,14,Michelle Taggart,2013-03-07 13:54:48.621,NaT,NaT,NaN,NaN,"[{account_id=280883, scanned_at=null, scanned_...",NaN,NaN,"{account_id=280883, scanned_at=null, scanned_t...",ticketmaster,ticketing,1615,True,<NA>,milb_aviators,EVLB0317,2013 Big League Weekend,Cashman Field,Cubs vs Rangers,2013-03-17,2013-03-17,SUN,13:05:00,Afternoon,Single Event,23,2013 Las Vegas 51s Season,2013,<NA>,1,1,0.0,False,Sports,Sports:mlb Pro Baseball,a8e6963060c210de3d77e73300399c5c,{ticketing=[280883]},<NA>,Mike,Rodriguez,<NA>,<NA>,<NA>,United States,<NA>,<NA>,<NA>,Personal,Michael Rodriguez,NaN,[],2023-03-23 18:43:28,"[{name=Las Vegas Ballpark, distance=null}]",280883,<NA>
1,ticketmaster,ticketing,15.0,milb_aviators,Group,54D9BB0D58E93216E0302AF0049CC16F,PZONE,GA0,718,23,1370,EVLB0822,<NA>,Party Zone 2,0,<NA>,390534,HGY,15.0,16,Erik Eisenberg,2013-08-15 10:27:17.761,NaT,NaT,NaN,NaN,"[{account_id=390534, scanned_at=null, scanned_...",NaN,NaN,"{account_id=390534, scanned_at=null, scanned_t...",ticketmaster,ticketing,1370,False,64,milb_aviators,EVLB0822,Las Vegas 51s Vs Fresno,Cashman Field,Las Vegas vs Fresno,2013-08-22,2013-08-22,THU,19:05:00,Evening,Single Event,23,2013 Las Vegas 51s Season,2013,64,1,1,0.0,False,Sports,Sports:minor League Baseball,d0a76874bc41970586d3f192905d56ec,"{ticketing=[390534, 513146]}",hghlvlchris@aol.com,Chris,Cryer,9355 Aster Pointe Ct,Las Vegas,Nevada,United States,89123,+1 (702) 375-4310,Fletcher Jones Smart Cars,Personal,Michelle Taggart,NaN,[],2023-03-23 18:43:28,"[{name=Las Vegas Ballpark, distance=20.6892010...",390534,<NA>
2,ticketmaster,ticketing,14.0,milb_aviators,Individual,E1FE4F782E514400B7F4D46FCAE656FD,P12,T,2,23,1362,EVLB0809,<NA>,Adult,0,<NA>,-1,C*,14.0,14,Michelle Taggart,2013-08-09 19:04:59.100,NaT,NaT,NaN,NaN,"[{account_id=-1, scanned_at=null, scanned_time...",NaN,NaN,"{account_id=-1, scanned_at=null, scanned_time=...",ticketmaster,ticketing,1362,False,56,milb_aviators,EVLB0809,Las Vegas 51s Vs Oklahoma City,Cashman Field,Las Vegas vs Oklahoma,2013-08-09,2013-08-09,FRI,19:05:00,Evening,Single Event,23,2013 Las Vegas 51s Season,2013,56,1,1,0.0,False,Sports,Sports:minor League Baseball,47d5208dde6ae6afc291ab209c579628,{ticketing=[-1]},<NA>,<NA>,Retail Sale No Account,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Personal,Michelle Taggart,NaN,[],2023-03-23 18:43:28,"[{name=Las Vegas Ballpark, distance=null}]",-1,<NA>
3,ticketmaster,ticketing,5.0,milb_aviators,Individual,6F688B6F48F7670F8EA411E4F85F0DD1,R4,O,3,23,1316,EVLB0427,<NA>,Adult,0,<NA>,-1,D*,5.0,14,Michelle Taggart,2013-04-27 12:09:30.130,NaT,NaT,NaN,NaN,"[{account_id=-1, scanned_at=null, scanned_time...",NaN,NaN,"{account_id=-1, scanned_at=null, scanned_time=...",ticketmaster,ticketing,1316,False,11,milb_aviators,EVLB0427,Las Vegas 51s Vs Tacoma,Cashman Field,Las Vegas vs Tacoma,2013-04-27,2013-04-27,SAT,19:05:00,Evening,Single Event,23,2013 Las Vegas 51s Season,2013,11,1,1,0.0,False,Sports,Sports:minor League Baseball,47d5208dde6ae6afc291ab209c579628,{ticketing=[-1]},<NA>,<NA>,Retail Sale No A

In [6]:
df_kernel = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """with 
        customers as (
            select c.id as scv_id, source_system_type, account_id, venue
            from integrations_milb_aviators.customers as c
            cross join unnest(account_ids) as t(source_system_type, source_account_ids)
            cross join unnest(source_account_ids) as a(account_id)
            cross join unnest(venues) as v(venue)
        ),
        ticketing_customers as (
            select 
                c.scv_id, 
                c.venue.distance as distance_to_venue, 
                te.season_year, 
                t.event_id,
                te.start_date as event_date,
                t.stlr_product, 
                t.seat_id,
                t.price,
                t.purchased_at,
                t.scans is not null as was_attended
            from customers as c
            join tickets as t on t.account_id = c.account_id and source_system_type = 'ticketing'
            join ticketing_events as te on te.id = t.event_id
        )

        select c.*, date_diff('day', first_purchase_date, CURRENT_DATE) as tenure
        from (
            select 
                scv_id, 
                distance_to_venue, 
                stlr_product, 
                season_year, 
                max(event_date) as last_event_date,
                count(*) as tickets_purchased, 
                sum(price) as total_spent, 
                (1.0 * sum(case when was_attended then 1 else 0 end)) / count(*) as attendance_percentage,
                max(case when was_attended then event_date else null end) as last_attendance_date,
                (lead(count(*), 1) over (partition by scv_id order by season_year) is not null) as is_next_year_buyer
            from ticketing_customers
            group by scv_id, distance_to_venue, stlr_product, season_year) as c
        join (
            select scv_id, min(purchased_at) as first_purchase_date
            from ticketing_customers
            group by scv_id
        ) as t
        on c.scv_id = t.scv_id
    """
)

df_kernel.shape

(201687, 11)

In [8]:
df_kernel.head(5)

,scv_id,distance_to_venue,stlr_product,season_year,last_event_date,tickets_purchased,total_spent,attendance_percentage,last_attendance_date,is_next_year_buyer,tenure
0,00005d638f4a29b761e274cd5610a15c,19.413923,Individual,2013,2013-05-13 00:00:00,2,28.0,1.0,2013-05-13 00:00:00,True,3614
1,00005d638f4a29b761e274cd5610a15c,19.413923,Individual,2022,2022-09-24 06:00:00,4,124.0,1.0,2022-09-24 06:00:00,False,3614
2,000306d7b9ba944964bc185463fdd6ea,544.084965,Group,2013,2013-05-31 00:00:00,1,14.0,0.0,NaT,False,3596
3,000587e1c86c0277ff9d85f5b9ca90c8,1.608480,Individual,2019,2019-06-21 00:00:00,2,54.0,1.0,2019-06-21 00:00:00,True,1445
4,000587e1c86c0277ff9d85f5b9ca90c8,1.608480,Individual,2021,2021-07-02 00:00:00,2,60.0,1.0,2021-07-02 00:00:00,False,1445


In [11]:
df_kernel_view = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        select * 
        from integrations_ds_milb_aviators.cohort_ticketing
    """
)
df_kernel_view.shape

(4864630, 15)

In [12]:
df_kernel_view.head(5)

,scv_id,season_year,event_id,game_number,event_date,account_id,product,price,revenue,purchased_at,returned_at,forwards,resales,scans,distance_to_venue
0,0a93cad85f4845d421964d9c1a42015d,2021,3788,<NA>,2021-07-24,1319588,Individual,40.0,40.0,2021-07-23 07:36:38.590,NaT,NaN,NaN,"[{account_id=1319588, scanned_at=null, scanned...",21.124010
1,fbcd5c8297894592feb1095b0d4d1ef0,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,4e48757ffd1b84464032f690eb067c68,2019,2757,34,2019-06-09,1487922,Individual,15.0,15.0,2019-05-26 20:00:46.210,NaT,NaN,NaN,"[{account_id=1487922, scanned_at=null, scanned...",26.377332
3,4e48757ffd1b84464032f690eb067c68,2019,2757,34,2019-06-09,1487922,Individual,15.0,15.0,2019-05-26 20:00:46.210,NaT,NaN,NaN,"[{account_id=1487922, scanned_at=null, scanned...",26.377332
4,4e48757ffd1b84464032f690eb067c68,2019,2757,34,2019-06-09,1487922,Individual,15.0,15.0,2019-05-26 20:00:46.210,NaT,NaN,NaN,NaN,26.377332


In [13]:
df_kernel_2 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """ with 
        customers as (
            select scv_id, distance_to_venue, min(purchased_at) as first_purchase_date
            from integrations_ds_milb_aviators.cohort_ticketing
            group by scv_id, distance_to_venue
        ),
        tickets as (
            select 
                scv_id, 
                product, 
                season_year, 
                event_date,
                price, 
                game_number,
                filter(scans, x -> x.account_id = account_id and x.is_valid) as owner_scans
            from integrations_ds_milb_aviators.cohort_ticketing
        ),
        games as (
            select 
                *,
                lead(game_number, 1) over (partition by scv_id, product, season_year order by game_number) as next_game_number
            from (
                select distinct
                    scv_id, 
                    product,
                    season_year,
                    game_number,
                    first_value(game_number) over (partition by product, season_year order by game_number desc) as max_game_number
                from tickets
            )
            where game_number is not null
        ),
        seasons as (
            select 
                t.scv_id, 
                t.product, 
                t.season_year, 
                max(t.event_date) as last_event_date,
                count(*) as tickets_purchased, 
                sum(t.price) as total_spent, 
                (1.0 * sum(case when t.owner_scans is not null and cardinality(t.owner_scans) > 1 then 1 else 0 end)) / count(*) as attendance_percentage,
                max(case when t.owner_scans is not null and cardinality(t.owner_scans) > 1 then event_date else null end) as last_attendance_date,
                (lead(count(*), 1) over (partition by t.scv_id order by t.season_year) is not null) as is_next_year_buyer,
                sum(
                    case when coalesce(g.next_game_number, g.max_game_number) - g.game_number > 1 then 1 else 0 end
                ) as consecutive_games_missed
            from tickets as t
            left join games as g on t.scv_id = g.scv_id and t.season_year = g.season_year and t.product = g.product
            group by t.scv_id, t.product, t.season_year
        )

        select 
            s.*,
            date_diff('day', c.first_purchase_date, s.last_event_date) as tenure,
            date_diff('day', s.last_event_date, localtimestamp) as recency,
            c.distance_to_venue,
            c.first_purchase_date
        from seasons as s
        left join games as g on s.scv_id = g.scv_id and s.season_year = g.season_year and s.product = s.product
        left join customers as c on s.scv_id = c.scv_id
    """
)
df_kernel_2.shape

(1462556, 14)

In [14]:
df_kernel_2.head(5)

,scv_id,product,season_year,last_event_date,tickets_purchased,total_spent,attendance_percentage,last_attendance_date,is_next_year_buyer,consecutive_games_missed,tenure,recency,distance_to_venue,first_purchase_date
0,00013e0223f9d6ce871a0371e736f859,Individual,2021,2021-09-24,5,135.0,0.0,NaT,False,5,0,557,367.000995,2021-09-24 15:49:51.080
1,0006f5f5aade62ba97405a18488bf007,Individual,2018,2018-09-01,4,48.0,0.0,NaT,False,4,0,1676,21.124010,2018-09-01 18:46:32.570
2,00095d1c2f2393ebf3d7009a3605bc42,<NA>,<NA>,NaT,1,NaN,0.0,NaT,False,0,<NA>,<NA>,NaN,NaT
3,000eb519bd71be78fd9a767228fd8768,Individual,2017,2017-07-04,36,576.0,0.0,NaT,False,36,62,2100,3.022490,2017-05-02 13:21:34.200
4,000eb519bd71be78fd9a767228fd8768,Individual,2017,2017-07-04,36,576.0,0.0,NaT,False,36,62,2100,3.022490,2017-05-02 13:21:34.200


In [ ]:
df_redshift = redshift_tools.get_retention_dataset(
    cluster = "prod-app",
    database =  "stlrmilb",
    lkupclientid = "15",
    start_year = 2010,
    end_year = 2023
)

df_redshift.shape

Authorized as AROASQ4JELIXYLYV6P4UV:gdonst@stellaralgo.com


(159283, 24)

In [ ]:
df_kernel.head()

,scv_id,distance_to_venue,stlr_product,season_year,last_event_date,tickets_purchased,total_spent,attendance_percentage,last_attendance_date,is_next_year_buyer,tenure
0,000d3732aecbb17de5443dd71fea2623,23.179834,Individual,2015,2015-09-05 00:00:00,1,14.0,1.000000,2015-09-05 00:00:00,True,2769
1,000d3732aecbb17de5443dd71fea2623,23.179834,Individual,2016,2016-08-27 00:00:00,6,85.0,1.000000,2016-08-27 00:00:00,False,2769
2,0011f620b54d21fad6b06d3f6fd6b89c,NaN,Individual,2022,2022-04-08 06:00:00,2,46.0,1.000000,2022-04-08 06:00:00,False,363
3,0013b2eeb0b15ac34461fb67f4d2b099,NaN,Individual,2022,2022-05-13 06:00:00,6,120.0,0.833333,2022-05-13 06:00:00,False,332
4,001a201f03ccf63a2684e723b11a5d8a,21.633013,Individual,2013,2013-05-24 00:00:00,4,40.0,1.000000,2013-05-24 00:00:00,False,3602


In [ ]:
df_redshift.head()

,lkupclientid,clientcode,dimcustomermasterid,year,productgrouping,totalspent,recentdate,attendancepercent,renewedbeforedays,source_tenure,tenure,disttovenue,recency,missed_games_1,missed_games_2,missed_games_over_2,forward_records,opentosendratio,clicktosendratio,clicktoopenratio,gender,phonecall,inperson_contact,isnextyear_buyer
0,15,VEGAS51S,739691640,2019,Group,14.0,1970-01-01,0.00,14,1460,14,17.40,0,1,0,0,0,0.00,0.00,0.00,Unknown,0,0,0
1,15,VEGAS51S,739446034,2019,Half Season,1100.0,2019-08-30,0.69,183,1825,328,3.35,1,3,3,1,8,0.00,0.00,0.00,Unknown,0,0,0
2,15,VEGAS51S,739641777,2016,Individual,41.0,2016-05-26,1.00,0,2555,49,19.15,0,0,0,0,0,0.00,0.00,0.00,Unknown,0,0,0
3,15,VEGAS51S,739692822,2019,Group,19240.0,2019-08-18,0.60,25,2190,835,10.92,0,0,0,1,0,0.00,0.00,0.00,Unknown,0,0,0
4,15,VEGAS51S,739580670,2022,Individual,75.0,1970-01-01,0.00,34,1460,1127,20.72,0,1,0,0,1,0.00,0.00,0.00,Unknown,0,0,0


In [ ]:
df_kernel.season_year.value_counts().sort_index()

2012        1
2013     8282
2014     9380
2015    11647
2016    12047
2017    13819
2018    13958
2019    39300
2020     6082
2021    36230
2022    41762
2023     9179
Name: season_year, dtype: Int64

In [ ]:
df_redshift.year.value_counts().sort_index()

2013     7692
2014     8619
2015    11169
2016    11403
2017    12859
2018    13271
2019    29934
2021    29520
2022    34816
Name: year, dtype: int64

In [ ]:
df_kernel["stlr_product"].value_counts()

Individual     155773
Group           24012
Comp             8896
Mini Plan        5365
Full Season      4539
Half Season      2321
Flex Plan         781
Name: stlr_product, dtype: Int64

In [ ]:
df_redshift["productgrouping"].value_counts()

Individual     128826
Group           22294
Mini Plan        3088
Full Season      2947
Half Season      1412
Flex Plan         716
Name: productgrouping, dtype: int64

In [ ]:
df_kernel["total_spent"].sum()

333108200.0

In [ ]:
df_redshift["totalspent"].sum()

43810838.449999996